In [27]:
import pandas as pd

data_path = "../data/"

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path+'test.csv')
submission = pd.read_csv(data_path+ 'sampleSubmission.csv')


In [28]:
train = train[train['weather'] != 4]

all_data = pd.concat([train,test], ignore_index=True)

In [29]:
all_data['datetime'] = pd.to_datetime(all_data['datetime'])

all_data['year'] = all_data['datetime'].dt.year
all_data['month'] = all_data['datetime'].dt.month
all_data['day'] = all_data['datetime'].dt.day
all_data['hour'] = all_data['datetime'].dt.hour
all_data['minute'] = all_data['datetime'].dt.minute
all_data['second'] = all_data['datetime'].dt.second
all_data['weekday'] = all_data['datetime'].dt.weekday
all_data.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour,minute,second,weekday
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3.0,13.0,16.0,2011,1,1,0,0,0,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8.0,32.0,40.0,2011,1,1,1,0,0,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5.0,27.0,32.0,2011,1,1,2,0,0,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3.0,10.0,13.0,2011,1,1,3,0,0,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0.0,1.0,1.0,2011,1,1,4,0,0,5


In [30]:
drop_features = ['casual', 'registered', 'datetime','month','windspeed']
all_data = all_data.drop(drop_features, axis =1)

In [31]:
X_train = all_data[~pd.isnull(all_data['count'])].drop(['count'],axis=1)
X_test = all_data[pd.isnull(all_data['count'])].drop(['count'],axis=1)

y=train['count']

In [32]:
import numpy as np

def rmsle(y_true, y_pred, convertExp=True):
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)

    log_true = np.nan_to_num(np.log(y_true+1))
    log_pred = np.nan_to_num(np.log(y_pred+1))

    output = np.sqrt(np.mean((log_true - log_pred)**2))
    return output


In [33]:
from sklearn.model_selection import GridSearchCV    #하이퍼 파라미터 확인해주는 함수
from sklearn import metrics

릿지

In [34]:
from sklearn.linear_model import Ridge
ridge_model = Ridge()

# max_iter: 최대 반복 횟수,  
ridge_params = {'max_iter': [3000], 'alpha': [0.1,1,2,3,4,10,30,100,200,300,400,800,900,1000]}

rmsle_score = metrics.make_scorer(rmsle, greater_is_better=False)
rmsle_score

make_scorer(rmsle, greater_is_better=False)

In [35]:
gridsearch_ridge_model = GridSearchCV(estimator = ridge_model,
                                      param_grid=ridge_params,
                                      scoring = rmsle_score,
                                      cv = 5)

In [36]:
log_y = np.log(y)
gridsearch_ridge_model.fit(X_train, log_y)

preds = gridsearch_ridge_model.best_estimator_.predict(X_train)

f'릿지 회귀 RMSLE 값 : {rmsle(log_y, preds, True):.4f}'

'릿지 회귀 RMSLE 값 : 1.0203'

라쏘

In [37]:
from sklearn.linear_model import Lasso

lasso_model = Lasso()

lasso_alpha = 1/np.array([0.1,1,2,3,4,10,30,100,200,300,400,800,900,1000])
lasso_params = {'max_iter': [3000], 'alpha': lasso_alpha}

gridsearch_lasso_model = GridSearchCV(estimator = lasso_model,
                                      param_grid = lasso_params,
                                      scoring = rmsle_score,
                                      cv = 5)

log_y = np.log(y)
gridsearch_lasso_model.fit(X_train, log_y)

preds = gridsearch_lasso_model.best_estimator_.predict(X_train)
f'라쏘 회귀 RMSLE 갑: {rmsle(log_y, preds, True):.4f}'

'라쏘 회귀 RMSLE 갑: 1.0203'

랜덤포레스트 회귀 모델

In [45]:
from sklearn.ensemble import RandomForestRegressor

randomforest_model = RandomForestRegressor()
rf_params = {'random_state':[42], 'n_estimators': [100,120,140]}
gridsearch_random_forest_model = GridSearchCV(estimator = randomforest_model, param_grid = rf_params,
                                              scoring=rmsle_score, cv = 5)
log_y = np.log(y)
gridsearch_random_forest_model.fit(X_train,log_y)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'n_estimators': [100, 120, 140], 'random_state': [42]},
             scoring=make_scorer(rmsle, greater_is_better=False))

In [49]:
preds = gridsearch_random_forest_model.best_estimator_.predict(X_train)
f'랜덤 포레스트 회귀 RMSLE 값 : {rmsle(log_y, preds, True):.4f}'

'랜덤 포레스트 회귀 RMSLE 값 : 0.1101'